In [2]:
import pandas as pd
import numpy as np

# Task1
df = pd.read_csv('data2.csv')

# Q1
print(df.isnull().sum())

df = df.dropna()

# Q2
print(df['tumor-size'].value_counts())
print(df['inv-nodes'].value_counts())

replace_values1 = {'14-Oct': '10-14', '9-May': '5-9'}
df['tumor-size'] = df['tumor-size'].replace(replace_values1)

replace_values2 = {'5-Mar': '3-5', '8-Jun': '6-8', '11-Sep': '9-11', '14-Dec': '12-14'}
df['inv-nodes'] = df['inv-nodes'].replace(replace_values2)

# Q3
excel = pd.read_excel('variables.xlsx', index_col=0)
temp1_variables = excel['Description'].tolist()
temp2_variables = [item.split(', ') for item in temp1_variables]
variables = []
for item in temp2_variables:
    for sublist in item:
        variables.append(sublist)

for column in df.columns[1:]:
    if column == 'irradiat':
        for i in df.index:
            first_index = variables.index(df.loc[i, column])
            second_index = variables[first_index+1:].index(df.loc[i, column]) + first_index + 1
            df.loc[i, column] = second_index
    else:
        for i in df.index:
            df.loc[i, column] = variables.index(str(df.loc[i, column]))
            
ind2val = {}

for attribute, row in excel.iterrows():
    if attribute != 'irradiat':
        for item in row['Description'].split(', '):
            index = variables.index(item)
            ind2val[index] = f"{attribute}={item}"
    else:
        for item in row['Description'].split(', '):
            first_index = variables.index(item)
            second_index = variables[first_index+1:].index(item) + first_index + 1
            index = second_index
            ind2val[index] = f"{attribute}={item}"
print(ind2val)

id             0
Class          0
age            0
menopause      0
tumor-size     0
inv-nodes      0
node-caps      8
deg-malig      0
breast         0
breast-quad    1
irradiat       0
dtype: int64
tumor-size
30-34     57
25-29     51
20-24     48
15-19     29
14-Oct    28
40-44     22
35-39     19
0-4        8
50-54      8
9-May      4
45-49      3
Name: count, dtype: int64
inv-nodes
0-2       209
5-Mar      34
8-Jun      17
11-Sep      7
15-17       6
14-Dec      3
24-26       1
Name: count, dtype: int64
{0: 'Class=no-recurrence-events', 1: 'Class=recurrence-events', 2: 'age=10-19', 3: 'age=20-29', 4: 'age=30-39', 5: 'age=40-49', 6: 'age=50-59', 7: 'age=60-69', 8: 'age=70-79', 9: 'age=80-89', 10: 'age=90-99', 11: 'menopause=lt40', 12: 'menopause=ge40', 13: 'menopause=premeno', 14: 'tumor-size=0-4', 15: 'tumor-size=5-9', 16: 'tumor-size=10-14', 17: 'tumor-size=15-19', 18: 'tumor-size=20-24', 19: 'tumor-size=25-29', 20: 'tumor-size=30-34', 21: 'tumor-size=35-39', 22: 'tumor-size=40-4

In [3]:
# Task2
# Q1

# Calulate the support rate of each itemset
def calculate_support(df, itemsets):
    support_rate = {}
    for itemset in itemsets:
        if itemset:
            num = df.apply(lambda row: all(item in set(row) for item in itemset), axis=1).sum()
            # 接受一个参数row，表示df的一行数据，检查itemset中的所有元素是否都在row中出现，若都出现则返回True，否则返回False
            support = num / len(df)
            support_rate[frozenset(itemset)] = support
    return support_rate

# Generate frequent k-itemsets
def generate_frequent(df, Candidate_k, min_support=0.4):
    Frequent_k = []
    support_rate = calculate_support(df, Candidate_k)
    for itemset in Candidate_k:
        if support_rate[frozenset(itemset)] >= min_support:
            Frequent_k.append(itemset)

    return Frequent_k

# Generate candidate k-itemsets
def generate_candidate(last_frequent, k):
    Candidate_k = []
    for i in range(len(last_frequent)):
        for j in range(i+1, len(last_frequent)):
            new_set = last_frequent[i] | last_frequent[j]
            if len(new_set) == k:
                all_subsets_in_last_frequent = True
                for item in new_set:# 检验new_set的所有子集是否都在last_frequent中
                    if new_set - {item} not in last_frequent:
                        all_subsets_in_last_frequent = False
                        break
                if all_subsets_in_last_frequent:        
                    Candidate_k.append(new_set)
    Candidate_k = list(set(map(frozenset, Candidate_k)))# 使用 map() 函数对 Candidate_k 列表中的每个元素都应用 frozenset() 函数
    return Candidate_k

frequent_itemsets = {}
k = 1
candidates = [{i} for i in range(len(variables))]

while True:
    frequent = [set(x) for x in generate_frequent(df, candidates)]
    if not frequent:
        break
    frequent_itemsets[k] = frequent
    print(f"Frequent {k}-itemsets: {frequent}")
    k += 1
    candidates = generate_candidate(frequent, k)

# Q2
# Generate new sets
def generate_sets(item_sets):
    new_sets = []
    for item_set in item_sets:
        if 0 in item_set:
            new_sets.append(frozenset(item_set))
            for item in item_set:
                if item == 0:
                    new_sets.append(frozenset({item}))
                    new_sets.append(frozenset(item_set - {item}))
    return new_sets


# Generate association rules
def genarate_rules(Frequent_k, df, min_cof=0.75):
    support_rate = calculate_support(df, generate_sets(Frequent_k))
    for item_set in Frequent_k:
        if 0 not in item_set:
            continue
        else:
            for item in item_set:
                if item == 0:                  
                    cof = support_rate[frozenset(item_set)] / support_rate[frozenset(item_set - {item})]
                    if cof < min_cof:
                        continue
                    else:
                        lift = cof / support_rate[frozenset([0])] 
                        print(f"{item_set-{item}}->{{{item}}}: cof = {cof}, lift = {lift}")

for k in list(frequent_itemsets.keys())[1:]:
    genarate_rules(frequent_itemsets[k], df)


Frequent 1-itemsets: [{0}, {12}, {13}, {26}, {40}, {42}, {44}, {45}, {52}]
Frequent 2-itemsets: [{40, 13}, {26, 13}, {40, 44}, {26, 44}, {0, 26}, {26, 52}, {0, 40}, {0, 52}, {40, 52}, {44, 52}, {52, 13}, {40, 26}]
Frequent 3-itemsets: [{0, 40, 52}, {40, 26, 52}, {0, 26, 52}, {0, 26, 40}]
Frequent 4-itemsets: [{0, 40, 26, 52}]
{26}->{0}: cof = 0.7942583732057417, lift = 1.122497802948931
{40}->{0}: cof = 0.7737556561085973, lift = 1.0935220241942931
{52}->{0}: cof = 0.7627906976744185, lift = 1.0780256288561936
{40, 52}->{0}: cof = 0.8074866310160427, lift = 1.1411928407726726
{26, 52}->{0}: cof = 0.8166666666666667, lift = 1.1541666666666668
{40, 26}->{0}: cof = 0.7999999999999999, lift = 1.1306122448979592
{40, 26, 52}->{0}: cof = 0.8238636363636364, lift = 1.1643378942486087
